# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
#your code here
import pandas as pd
import numpy as np




# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here
df=pd.read_csv(r"data\Admission_Predict.csv")
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [3]:
# your code here
df.columns = df.columns.str.strip()
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [4]:
# your code here
df.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
# your code here
df.isnull().any()

Serial No.           False
GRE Score            False
TOEFL Score          False
University Rating    False
SOP                  False
LOR                  False
CGPA                 False
Research             False
Chance of Admit      False
dtype: bool

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
# your code here
df.set_index("Serial No.")

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
# your code here
df.loc[:, ["GRE Score","CGPA"]]

,GRE Score,CGPA
0,337,9.65
1,316,8.00
2,322,8.67
3,314,8.21
4,330,9.34
...,...,...
380,324,9.04
381,325,9.11
382,330,9.45
383,312,8.78


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [8]:
# your code here

df.reset_index(inplace=True)
df

,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...,...
380,380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,383,384,312,103,3,3.5,4.0,8.78,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [9]:
# your code here
df=df.loc[df['CGPA'] > 9]
df

,index,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,0,1,337,118,4,4.5,4.5,9.65,1,0.92
4,4,5,330,115,5,4.5,3.0,9.34,1,0.90
10,10,11,328,112,4,4.0,4.5,9.10,1,0.78
19,19,20,328,116,5,5.0,5.0,9.50,1,0.94
20,20,21,334,119,5,5.0,4.5,9.70,1,0.95
...,...,...,...,...,...,...,...,...,...,...
379,379,380,329,111,4,4.5,4.0,9.23,1,0.89
380,380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,382,383,330,116,4,5.0,4.5,9.45,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [10]:
# your code here
df=df.loc[df['SOP'] > 3.5]
df
df["Chance of Admit"].mean()

0.8945263157894737

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [11]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    return (col - np.mean(col))/np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [12]:
# your code here
CGPA_std = standardize(df['CGPA'])
GRE_std = standardize(df['GRE Score'])
admissions = pd.DataFrame({'CGPA_std':CGPA_std, 'GRE_std' : GRE_std})
admissions


,CGPA_std,GRE_std
0,1.193452,1.139084
4,-0.059566,0.052297
10,-1.029645,-0.258214
19,0.587153,-0.258214
20,1.395551,0.673318
...,...,...
370,1.880591,0.828573
377,-0.948805,-0.568725
379,-0.504185,-0.102959
382,0.385053,0.052297


We will generate the decision choice at random using the code below. Please run the cell.

In [13]:
# Libraries
from random import choices

In [14]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [15]:
# your code here
admissions['deciding_column'] = df.lookup(admissions.index, decision_choice)
admissions.head(3)

C:\Users\DELL\AppData\Local\Temp\ipykernel_8872\421577964.py:2: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admissions['deciding_column'] = df.lookup(admissions.index, decision_choice)


KeyError: 'GRE_std'

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here
df['decision'] = np.where(df['deciding_column'] > 0.8, 1, 0)
df

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
# your code here\
df['decision'].sum()

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here
import re
df.columns

Index(['level_0', 'GRE Score', 'CGPA', 'index', 'Serial No.', 'TOEFL Score',
       'University Rating', 'SOP', 'LOR', 'Research', 'Chance of Admit'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here
df.columns = df.columns.str.replace('\W','').str.replace(' ','_').str.lower()
df.columns

C:\Users\DELL\AppData\Local\Temp\ipykernel_8500\309974410.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('\W','').str.replace(' ','_').str.lower()


Index(['level_0', 'grescore', 'cgpa', 'index', 'serialno', 'toeflscore',
       'universityrating', 'sop', 'lor', 'research', 'chanceofadmit'],
      dtype='object')